# Generate Mock Data

In [1]:
import os
import random
import numpy as np
import sys

import database.config as config
import mock_data.mock_data_utils as utils

### (1) Generate Spike Times

Specify seed information for mock spike trains:

In [2]:
patient_id = 1
session_nr = 1

nr_units_patient_1 = 100
begin_recording_time = 449860058000
stop_recording_time =  455889058000
nr_units_per_brain_region_1 = [7, 12, 9, 10, 15, 13, 8, 11, 12, 3]

Generate mock spikes for one "patient":

In [3]:
utils.generate_spikes(patient_id, session_nr, nr_units_patient_1, begin_recording_time, stop_recording_time)

### (2) Create Channel Names File

Channels names indicate which brain region a given unit was recorded from. 

Here, channel files consist of a look-up table used to determine where a spike train originates. Once in the database, the region labels allow for dynamically switching between region-specific analyses.

In [4]:
utils.generate_channel_file(patient_id, session_nr, nr_units_patient_1, nr_units_per_brain_region_1)

### (3) and (4) Create Events File and timedDAQ log:

Generates mock Events.nev file, a proprietary file format from Neuralynx. 

Necessary for matching spike train data to stimulus information. 

In conjunction with the DAQ file, allows a linear mapping between the two different timescales (Neural recording device time and local computer time) so that neural events can be matched to stimulus evvents and vice versa.

In [5]:
utils.make_events_and_daq(patient_id, session_nr, begin_recording_time, stop_recording_time)

Recording length, in usec:  6029000000
End time of recording, in epoch time:  1590534256608515
Length of interval iteratively added:  1127970


### (4) Create Watch Log File

Using the same seed time as used for the timedDAQ file, generate a watchlog file for a given "patient":

In [6]:
#utils.generate_perfect_watchlog(patient_id, session_nr)
utils.generate_playback_artifacts(patient_id, session_nr)

Length of the stimulus, in usec:  4728999960.0
End of stimulus, in epoch time:  1590532956608475.0
Length of interval iteratively added :  37613


### Repeat (1 - 4) for more "patients":

In [7]:
# Set seed info
patient_id = 2
session_nr = 1
nr_units_patient_2 = 84 
begin_recording_time = 349871349000
stop_recording_time  = 355900349000
nr_units_per_brain_region_2 = [12, 9, 10, 5, 7, 11, 8, 7, 12, 3]

# Run generators
utils.generate_spikes(patient_id, session_nr, nr_units_patient_2, begin_recording_time, stop_recording_time)
utils.generate_channel_file(patient_id, session_nr, nr_units_patient_2, nr_units_per_brain_region_2)
utils.make_events_and_daq(patient_id, session_nr, begin_recording_time, stop_recording_time)
utils.generate_playback_artifacts(patient_id, session_nr)

Recording length, in usec:  6029000000
End time of recording, in epoch time:  1590534256608515
Length of interval iteratively added:  1355440
Length of the stimulus, in usec:  4728999960.0
End of stimulus, in epoch time:  1590532956608475.0
Length of interval iteratively added :  37613


In [8]:
# Set seed info
patient_id = 3
session_nr = 1
nr_units_patient_3 = 59
begin_recording_time = 248860058000
stop_recording_time = 254889058000
nr_units_per_brain_region_3 = [6, 5, 4, 6, 7, 10, 3, 7, 5, 6]

# Run generators
utils.generate_spikes(patient_id, session_nr, nr_units_patient_2, begin_recording_time, stop_recording_time)
utils.generate_channel_file(patient_id, session_nr, nr_units_patient_2, nr_units_per_brain_region_2)
utils.make_events_and_daq(patient_id, session_nr, begin_recording_time, stop_recording_time)
utils.generate_playback_artifacts(patient_id, session_nr)

Recording length, in usec:  6029000000
End time of recording, in epoch time:  1590534256608515
Length of interval iteratively added:  1373348
Length of the stimulus, in usec:  4728999960.0
End of stimulus, in epoch time:  1590532956608475.0
Length of interval iteratively added :  37613


### (5) Create Stimulus Meta-Data 

For the purposes of demonstration, the meta-data consists of frames/movie time points in which a movie character was on-screen. 

Since the stimulus meta-data is taken only from the stimulus, this information only needs to be generated once per "experimental" paradigm. 

In [2]:
nr_movie_frames = 125725      # movie length: 5029 seconds (AVI file); 5029/0.04 = 125725
perfect_pts = [round((x * 0.04), 2) for x in range(1, nr_movie_frames+1)]  

annotator_ids = []
for i in range(len(config.annotators)):
    annotator_ids.append(config.annotators[i]['annotator_id'])

path_to_movie_annotations = "mock_data/movie_annotation"

if not os.path.exists(os.path.join(config.PATH_TO_REPO, path_to_movie_annotations)):
    os.makedirs(os.path.join(config.PATH_TO_REPO, path_to_movie_annotations))

nr_character_labels = 2

start_times_1 = [0, 5000.04, 7000.04, 12000.04]
stop_times_1 = [5000,7000,12000,12575]
values_1 = [1,0,1,0]
character1 = np.array([values_1, start_times_1, stop_times_1]) 
np.save("{}/{}/{}_character1_{}_20191212_character.npy".format(
    config.PATH_TO_REPO, path_to_movie_annotations, 1, random.choice(annotator_ids)), character1)


start_times_2 = [0, 400.04, 4000.04, 10000.04, 10500.04]
stop_times_2 = [400,4000,10000,10500,12575]
values_2 = [0,1,0,1,0]
character2 = np.array([values_2, start_times_2, stop_times_2]) 
np.save("{}/{}/{}_character2_{}_20191010_character.npy".format(
    config.PATH_TO_REPO, path_to_movie_annotations, 2, random.choice(annotator_ids)), character2)

start_times_3 = [0, 100.04, 500.04]
stop_times_3 = [100, 500, 12575]
values_3 = [0,1,0]
annot4 = np.array([values_3, start_times_3, stop_times_3]) 
np.save("{}/{}/{}_location1_{}_20200101_location.npy".format(
    config.PATH_TO_REPO, path_to_movie_annotations, 3, random.choice(annotator_ids)), annot4)
